Import libraries for data cleaning 

In [17]:
import csv
from tokenize import String

from notebooks.db_connection import get_db_engine
from notebooks.db_connection import execute_query
import numpy as np
import pandas as pd
import seaborn as sns
import os
import re

Reading CSV file

In [18]:
df_countries = pd.read_csv(r"C:\Users\ricca\Desktop\CSV\countries.csv\countries.csv")

Top records of CSV file: countries.csv

In [19]:
df_countries

,id,country
0,1000001,UK
1,1000001,USA
2,1000002,South Korea
3,1000003,USA
4,1000004,Germany
...,...,...
693471,1941593,China
693472,1941594,USA
693473,1941595,USA
693474,1941596,China


Checking for total dimension of the csv

In [20]:
df_countries.shape

(693476, 2)

Rename 'id' column in 'id_country'. "inplace = True" modifies directly the df without creating a copy

In [21]:
df_countries.rename(columns = {"id":"id_movie"} , inplace = True)

In [22]:
df_countries.columns

Index(['id_movie', 'country'], dtype='object')

Check info about column type

In [23]:
df_countries.dtypes

id_movie     int64
country     object
dtype: object

Correction of data type of the records

In [24]:
df_countries['id_movie'] = df_countries['id_movie'].astype('Int64')
df_countries['country'] = df_countries['country'].fillna(pd.NA).astype('string')
df_countries.dtypes

id_movie             Int64
country     string[python]
dtype: object

Check if there are duplicates

In [25]:
df_countries.duplicated().sum()

np.int64(0)

In [26]:
df_countries.drop_duplicates(inplace = True)
df_countries.duplicated().sum()

np.int64(0)

Check the missing values

In [27]:
df_countries.isnull().sum()

id_movie    0
country     0
dtype: int64

Dropping records with missing value 

In [28]:
df_countries = df_countries.dropna()
df_countries.isnull().sum()

id_movie    0
country     0
dtype: int64

Remove countries where id_movie doesn't exist

In [29]:
valid_movies = pd.read_csv("movies_valid_ids.csv")
df_countries = df_countries[df_countries["id_movie"].isin(valid_movies["id_movie"])]
df_countries.shape

(693475, 2)

Connecting to PostgreSQL database and population of tables

In [30]:
engine = get_db_engine()
sql = \
"""CREATE TABLE IF NOT EXISTS countries (
        id BIGSERIAL PRIMARY KEY,
        id_movie INTEGER CHECK (id_movie>0),
        country TEXT CHECK (country<>'')
        );
"""
execute_query(sql)

Query Executed Successfully!


Population of the table using .to_sql function

In [31]:
df_countries.to_sql("countries", engine, if_exists="append", index=False)

475